@Diksha. This notebook will

- Generate some simulated click times and trial durations
- Generate some simulated choices from a model based on a set of generative parameters
- Maximize the likelihood of the parameters given the data
- Compute the Hessian of the ML parameters
- Verify that the generative parameters can be recovered

In [1]:
addprocs(48);

In [2]:
using PyPlot, StatsBase, JLD, Pandas, DataFrames, Distributions
@everywhere using choice_observation, latent_DDM_common_functions, helpers

In [56]:
#some constants
dt,n,map_str,N,c,ntrials = 1e-2,53,"exp",2,5.,Int(5e4)

(0.01, 53, "exp", 2, 5.0, 50000)

In [57]:
#parameters of the latent model
pz = DataFrames.DataFrame(generative = vcat(1e-6,10.,-0.5,40.,1.,0.8,0.02), 
    name = vcat("σ_i","B", "λ", "σ_a","σ_s","ϕ","τ_ϕ"),
    fit = vcat(falses(1),trues(4),trues(2)),
    initial = vcat(1.,5.,-5,100.,2.,0.2,0.08));

#initialize any parameter that is not going to be fit to its generative value
pz[:initial][pz[:fit] .== false] = pz[:generative][pz[:fit] .== false];

In [58]:
#parameters for the choice observation
pd = DataFrames.DataFrame(generative = 0.1, name = "bias", fit = true, initial = 0.);
#ammend to the latent parameter dataframe
pchoice = vcat(pz,pd);

In [59]:
#generate some simulated clicks times and trial durations
data = latent_DDM_common_functions.sample_clicks(ntrials,dt);

In [60]:
#simulate choices from the model given the generative parameters
choice_observation.sampled_dataset!(data,pchoice[:generative]; num_reps=1, rng=4);

In [61]:
#compute the likelihood of the data given the generative parameters
choice_observation.do_LL(pchoice[:generative],dt,data,n)

-11794.359448028483

In [ ]:
#find the ML parameters
pchoice[:final],opt_output,state = choice_observation.do_optim(copy(pchoice[:initial]),
    pchoice[:fit],dt,data,n;map_str=map_str);

Iter     Function value   Gradient norm 
     0     2.732715e+04     5.231976e+03
     1     1.972505e+04     1.510253e+04
     2     1.506637e+04     3.192668e+04
     3     1.336507e+04     1.675301e+04
     4     1.260324e+04     2.423752e+03
     5     1.198576e+04     1.723058e+03
     6     1.197915e+04     1.210649e+03
     7     1.197543e+04     1.193942e+03
     8     1.193083e+04     3.243761e+03
     9     1.182682e+04     3.171932e+02
    10     1.181540e+04     9.849784e+02


In [ ]:
#compute the Hessian around the ML parameters
pchoice[:CI_z_plus], pchoice[:CI_z_minus], H = choice_observation.do_H(pchoice[:final],pchoice[:fit],
    dt,data,n;map_str=map_str)

#identify which ML parameters have generative parameters within the CI 
pchoice[:within_bounds] = (pchoice[:CI_z_minus] .< pchoice[:generative]) .& (pchoice[:CI_z_plus] .> pchoice[:generative]);

In [ ]:
pchoice